# Lab 2: Building LangChain Agents

**Module 2 - AI Agents and Framework Implementation**

| Duration | Difficulty | Framework | Exercises |
|----------|------------|-----------|----------|
| 90 min | Intermediate | LangChain | 4 |

## Learning Objectives

- Build custom tools for LangChain agents
- Implement a ReAct agent from scratch
- Create multi-tool agents
- Add custom callbacks for monitoring

## Setup

In [ ]:
# Install dependencies
# !pip install langchain langchain-openai python-dotenv

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.agents import tool, create_react_agent, AgentExecutor
from langchain.tools import Tool, StructuredTool
from langchain_core.prompts import PromptTemplate
from langchain.callbacks.base import BaseCallbackHandler
from pydantic import BaseModel, Field
from typing import Optional

# Set your API key
os.environ["OPENAI_API_KEY"] = "your-api-key-here"  # Replace with your key

# Initialize LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

---

## Exercise 1: Create a Calculator Tool

Build a simple calculator tool that can perform basic math operations.

**Your Task:** Complete the calculator tool implementation.

In [ ]:
@tool
def calculator(expression: str) -> str:
    """
    Evaluate a mathematical expression.
    Use this tool for any math calculations.
    Input should be a valid mathematical expression like '2 + 2' or '(5 * 3) - 10'.
    """
    # TODO: Safely evaluate the mathematical expression
    # Hint: Use eval() with caution, or implement a safer parser
    
    try:
        # Your code here
        result = None
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

In [ ]:
# Test your calculator tool
print(calculator.invoke("2 + 2"))
print(calculator.invoke("(10 * 5) / 2"))
print(calculator.invoke("2 ** 8"))

---

## Exercise 2: Create a Weather Tool with Structured Input

Build a weather lookup tool using Pydantic for structured input validation.

**Your Task:** Complete the weather tool with proper input schema.

In [ ]:
# Define input schema
class WeatherInput(BaseModel):
    """Input schema for weather lookup."""
    location: str = Field(description="The city name to get weather for")
    units: Optional[str] = Field(default="celsius", description="Temperature units: celsius or fahrenheit")


def get_weather(location: str, units: str = "celsius") -> str:
    """
    Get the current weather for a location.
    This is a mock implementation - in production, call a real weather API.
    """
    # TODO: Implement weather lookup (mock data for this exercise)
    # Return a formatted string with weather information
    
    # Mock weather data
    mock_weather = {
        "london": {"temp_c": 15, "condition": "Cloudy"},
        "new york": {"temp_c": 22, "condition": "Sunny"},
        "tokyo": {"temp_c": 28, "condition": "Humid"},
        "paris": {"temp_c": 18, "condition": "Partly Cloudy"},
    }
    
    # Your code here
    pass


# Create the structured tool
weather_tool = StructuredTool.from_function(
    func=get_weather,
    name="weather",
    description="Get current weather for a city. Provide location and optionally units (celsius/fahrenheit).",
    args_schema=WeatherInput
)

In [ ]:
# Test your weather tool
print(weather_tool.invoke({"location": "London"}))
print(weather_tool.invoke({"location": "Tokyo", "units": "fahrenheit"}))

---

## Exercise 3: Build a Multi-Tool ReAct Agent

Combine your tools into a ReAct agent that can reason and act.

**Your Task:** Create and configure the agent executor.

In [ ]:
# ReAct prompt template
REACT_PROMPT = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}"""

prompt = PromptTemplate.from_template(REACT_PROMPT)

In [ ]:
# TODO: Create the agent with your tools
tools = None  # Your code here - list of tools

# TODO: Create the ReAct agent
agent = None  # Your code here

# TODO: Create the agent executor
agent_executor = None  # Your code here

In [ ]:
# Test your agent with various queries
queries = [
    "What is 25 * 4 + 100?",
    "What's the weather like in London?",
    "If it's 15 degrees in London and 28 degrees in Tokyo, what's the temperature difference?"
]

for query in queries:
    print(f"\nQuery: {query}")
    print("-" * 50)
    # result = agent_executor.invoke({"input": query})
    # print(f"Answer: {result['output']}")

---

## Exercise 4: Implement Custom Callbacks

Create a custom callback handler to monitor agent execution.

**Your Task:** Implement the callback methods.

In [ ]:
class AgentMonitorCallback(BaseCallbackHandler):
    """Custom callback handler for monitoring agent execution."""
    
    def __init__(self):
        self.steps = []
        self.total_tokens = 0
    
    def on_llm_start(self, serialized, prompts, **kwargs):
        """Called when LLM starts processing."""
        # TODO: Log that LLM is starting
        pass
    
    def on_llm_end(self, response, **kwargs):
        """Called when LLM finishes."""
        # TODO: Track token usage if available
        pass
    
    def on_tool_start(self, serialized, input_str, **kwargs):
        """Called when a tool starts executing."""
        # TODO: Log tool name and input
        tool_name = serialized.get("name", "unknown")
        pass
    
    def on_tool_end(self, output, **kwargs):
        """Called when a tool finishes."""
        # TODO: Log tool output
        pass
    
    def on_agent_action(self, action, **kwargs):
        """Called when agent decides on an action."""
        # TODO: Record the action
        self.steps.append({
            "type": "action",
            "tool": action.tool,
            "input": action.tool_input
        })
    
    def on_agent_finish(self, finish, **kwargs):
        """Called when agent completes."""
        # TODO: Log final output
        pass
    
    def get_summary(self):
        """Return execution summary."""
        return {
            "total_steps": len(self.steps),
            "steps": self.steps,
            "tokens_used": self.total_tokens
        }

In [ ]:
# Test with callbacks
callback = AgentMonitorCallback()

# Recreate executor with callbacks
# agent_executor_with_callbacks = AgentExecutor(
#     agent=agent,
#     tools=tools,
#     verbose=True,
#     callbacks=[callback]
# )

# Run a query
# result = agent_executor_with_callbacks.invoke({"input": "What is 15 * 8 and what's the weather in Paris?"})

# Get summary
# print("\nExecution Summary:")
# print(callback.get_summary())

---

## Checkpoint

Congratulations! You've completed Lab 2. You should now understand:

- How to create custom tools with the `@tool` decorator
- How to use Pydantic for structured tool inputs
- How the ReAct pattern enables reasoning + acting
- How to monitor agent execution with callbacks

**Next:** Lab 3 - Advanced Agent Patterns